In [ ]:
!nvidia-smi

Sun Nov  1 02:37:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 載入函數

In [ ]:
import numpy as np
import tensorflow as tf
import os 

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

os.chdir('/content/drive/Shared drives/類技術班教材/標準版/ 深度學習基礎/DL_part1/Answer')
print(tf.__version__)

2.3.0


In [ ]:
# tensorflow.keras.utils.plot_model 所需安裝的 package
! pip install --user graphviz==0.8.2
! pip install --user pydot==1.4.1
! pip install --user pydot-ng==2.0.0

# 動手試試看 (三元一次方程式)

通常資料和神經網路內部訓練的維度是非常高的，所以通常不可能像上面兩個例子畫得出來，只能從評分指標 (MSE, R2, Precision, Recall) 去觀察模型表現。

請使用以下的資料點 $x$ 及 $y$ 去擬合函數：

$$
f(x,y,z) = 10x+8y+6z+4
$$

In [ ]:
# 準備資料, 共四個維度 f(x,y,z), x, y, z
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
z = np.linspace(0, 1, 100)
x_in = np.stack([x, y, z], axis=1)

y_true = 10 * x_in[:,0] + 8 * x_in[:,1] + 6 * x_in[:,2] + 4

noise = np.random.normal(0, 0.1, size=len(x))
y_noise = y_true + noise

In [ ]:
print('x_in_shape:', x_in.shape)
print('y_true_shape:', y_true.shape)
print('y_noise_shape:', y_noise.shape)

x_in_shape: (100, 3)
y_true_shape: (100,)
y_noise_shape: (100,)


## 建立模型

接下來使用 Functional API 的方式建立模型。

In [ ]:
input = tf.keras.layers.Input(shape=(3,), name='Input_layer')
dense = tf.keras.layers.Dense(1, use_bias=True, activation='linear', name='dense1', kernel_initializer='glorot_uniform')

output = dense(input)

model = tf.keras.Model(input, output, name='linear_function')

model.summary()

Model: "linear_function"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     [(None, 3)]               0         
_________________________________________________________________
dense1 (Dense)               (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


### 查看模型參數

使用 `.get_weights()` 可以拿取模型所有的參數

In [ ]:
w1_pred = model.get_weights()[0][0]
w2_pred = model.get_weights()[0][1]
w3_pred = model.get_weights()[0][2]
b_pred = model.get_weights()[1]

print('w1_true:', 10, '\nw2_true:', 8, '\nw1_true:', 6, '\nb_true:', 4)
print('-'*10)
print('w1_pred:', w1_pred, '\nw2_pred:', w2_pred, '\nw3_pred:', w3_pred, '\nb_pred:', b_pred)

w1_true: 10 
w2_true: 8 
w1_true: 6 
b_true: 4
----------
w1_pred: [0.8708621] 
w2_pred: [-0.5605281] 
w3_pred: [-0.365839] 
b_pred: [0.]


## 訓練模型

Tensorflow 最常使用來編譯模型的方式是透過 `.compile` 來指定損失函數 (loss function) 、優化器 (optimizer)、評估指標 (metrics) 以及 callbacks。

* Opimitizer 請使用 `SGD`, `learning_rate` 請設定為 `1e-2`
* Loss 請使用 `mean_squared_error`
* epochs 大小請設定 `300`

In [ ]:
sgd = tf.keras.optimizers.SGD(learning_rate=1e-2)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
# 訓練模型
model.fit(x_in, y_noise, epochs=300)  # 一個 epoch 會看過一次所有的資料

Epoch 1/300
4/4 [==============================] - 0s 3ms/step - loss: 282.8920
Epoch 2/300
4/4 [==============================] - 0s 2ms/step - loss: 208.7223
Epoch 3/300
4/4 [==============================] - 0s 2ms/step - loss: 155.6553
Epoch 4/300
4/4 [==============================] - 0s 2ms/step - loss: 119.2941
Epoch 5/300
4/4 [==============================] - 0s 2ms/step - loss: 89.3181
Epoch 6/300
4/4 [==============================] - 0s 2ms/step - loss: 64.8158
Epoch 7/300
4/4 [==============================] - 0s 2ms/step - loss: 47.6500
Epoch 8/300
4/4 [==============================] - 0s 2ms/step - loss: 37.0111
Epoch 9/300
4/4 [==============================] - 0s 2ms/step - loss: 28.9043
Epoch 10/300
4/4 [==============================] - 0s 2ms/step - loss: 22.9325
Epoch 11/300
4/4 [==============================] - 0s 2ms/step - loss: 17.8681
Epoch 12/300
4/4 [==============================] - 0s 2ms/step - loss: 14.6476
Epoch 13/300
4/4 [===========================

## 訓練完的預測表現

請使用 `mean_squared_error` 評估模型表現。

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(x_in)
mse = mean_squared_error(y_pred=y_pred, y_true=y_true)
print('MSE: %.4f' % mse)

MSE: 0.0096


### 查看訓練參數

使用 `.get_weights()` 可以拿取模型所有的參數

In [ ]:
w1_pred = model.get_weights()[0][0]
w2_pred = model.get_weights()[0][1]
w3_pred = model.get_weights()[0][2]
b_pred = model.get_weights()[1]

print('w1_true:', 10, '\nw2_true:', 8, '\nw1_true:', 6, '\nb_true:', 4)
print('-'*10)
print('w1_pred:', w1_pred, '\nw2_pred:', w2_pred, '\nw3_pred:', w3_pred, '\nb_pred:', b_pred)

w1_true: 10 
w2_true: 8 
w1_true: 6 
b_true: 4
----------
w1_pred: [8.781493] 
w2_pred: [7.3501062] 
w3_pred: [7.544795] 
b_pred: [4.18726]


In [ ]:
# 直接利用權重算出預測值，和 tensorflow 的預測結果比較
y_pred_numpy = x_in[:,0] * w1_pred + x_in[:,1] * w2_pred + x_in[:,2] * w3_pred + b_pred

print('y_pred_tensorflow:', y_pred[0:5])
print('y_pred_numpy: ', y_pred_numpy[:5])

y_pred_tensorflow: [[4.18726  ]
 [4.4264154]
 [4.665571 ]
 [4.9047265]
 [5.1438823]]
y_pred_numpy:  [4.18726015 4.42641565 4.66557115 4.90472665 5.14388215]
